<a href="https://colab.research.google.com/github/Rishav-hub/Challenge1/blob/main/Documentation_5_BatchNormalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Batch Normalization
- Although after changing various Activation function and initializing weights in different ways its not guareented that Vanishing and Exploding Gradient problems won't come back. 
- So, In a 2015 paper, Sergey Ioffe and Christian Szegedy proposed a technique called Batch Normalization (BN).
- This involves adding an operation in the model just before or after the activation function of each hidden layers.
- Simply zero centering and normalizing each input(), then scaling and shifting the results using two new parameters per layers.
- Parameters $\mu_{B} \Rightarrow$ Batch- Mean, $\sigma^{2}_{B}\Rightarrow$Batch Variance are non - trainable
- Parameters $\gamma \Rightarrow$ Scaling, $\beta\Rightarrow$Shifting are trainable

Steps - :

- Mini Batch Mean $\Rightarrow \mu_{B} = \frac{1}{M_{B}}\displaystyle\sum\limits_{i=0}^n X^{i}$
- Mini - Batch Variance $\Rightarrow \sigma^{2}_{B} = \frac {1}{M_{B}}\displaystyle\sum\limits_{i=0}^n (X^{i} - \mu_{B})$
- Normalizing $X^{i} \Rightarrow$ $\hat{X}^{i} = \frac{X^{i} - \mu_{B}}{\sqrt{\sigma^{2}_{B}} + \epsilon}$, $\epsilon \Rightarrow $ smoothning term $10^{-7}$ to avoid zero division error.
- $$Z^{i} = \gamma\hat{X} + \beta$$

### Two types of approach - 
- Batch Normalization layer before the activation function.
- Batch Normalization layer after the activation function.



#### Batch Normalization layer before the activation function
A shifted output that lies within the trainable area. This will give the activation function a better input in his trainable part.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
minist = tf.keras.datasets.mnist

(X_train_full, y_train_full), (X_test, y_test) = minist.load_data()

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train =  y_train_full[:5000], y_train_full[5000:]

In [ ]:
model = tf.keras.models.Sequential([
 tf.keras.layers.Flatten(input_shape=[28, 28]),
 tf.keras.layers.BatchNormalization(),
 tf.keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal", use_bias=False),
 tf.keras.layers.BatchNormalization(),
 tf.keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal", use_bias=False),
 tf.keras.layers.BatchNormalization(),
 tf.keras.layers.Dense(10, activation="softmax")
])


#### Why did we remove the bias terms in the Dense layers ?
Since a Batch Normalization layer includes one offset parameter per input, we can remove the bias term from the previous layer.

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235200    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30000     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

In [ ]:
784 * 4 # mean, variance, gamma and Beta

3136

Combining all BN layers 

In [ ]:
784 * 4 + 300*4+ 100*4

4736

In [ ]:
4736//2

2368

In [ ]:
271346 - 2368

268978

### Why did we divide the total BN layers by 2 ?
This is beacause as we can see from the model.summary() only 268,978 are trainable from total 271,346 parameters, this is because we only make $\gamma    and  \beta $as trainable parameter. So, to exclude $\mu and \sigma$ we divide it by 2.

In [ ]:
for variable in model.layers[1].variables:
  print(variable.name, variable.trainable)

batch_normalization/gamma:0 True
batch_normalization/beta:0 True
batch_normalization/moving_mean:0 False
batch_normalization/moving_variance:0 False


In [ ]:
LOSS_FUNCTION = "sparse_categorical_crossentropy" # use => tf.losses.sparse_categorical_crossentropy
OPTIMIZER = tf.keras.optimizers.SGD(learning_rate=1e-3) # or use with custom learning rate=> tf.keras.optimizers.SGD(0.02)
METRICS = ["accuracy"]

model.compile(loss=LOSS_FUNCTION,
              optimizer=OPTIMIZER,
              metrics=METRICS)

In [ ]:
EPOCHS = 10
VALIDATION_SET = (X_valid, y_valid)

new_history = model.fit(X_train, y_train, epochs=EPOCHS,
                    validation_data=VALIDATION_SET,batch_size = 32)

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.8477 - accuracy: 0.7419 - val_loss: 0.4948 - val_accuracy: 0.8578
Epoch 2/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4685 - accuracy: 0.8631 - val_loss: 0.3934 - val_accuracy: 0.8844
Epoch 3/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3987 - accuracy: 0.8831 - val_loss: 0.3460 - val_accuracy: 0.8990
Epoch 4/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3630 - accuracy: 0.8926 - val_loss: 0.3170 - val_accuracy: 0.9084
Epoch 5/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3348 - accuracy: 0.9017 - val_loss: 0.2945 - val_accuracy: 0.9154
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3185 - accuracy: 0.9050 - val_loss: 0.2804 - val_accuracy: 0.9202
Epoch 7/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3026 - accuracy: 0.9106 - val_loss: 0.2682 - val_accuracy:

In [ ]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 43.6155 - accuracy: 0.7975


[43.61552047729492, 0.7975000143051147]

#### Batch Normalization layer after the activation function

In [ ]:
LAYERS_BN_BIAS_FALSE = [
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(100, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(10, activation="softmax")
]

model_2 = tf.keras.models.Sequential(LAYERS_BN_BIAS_FALSE)

LOSS_FUNCTION = "sparse_categorical_crossentropy" # use => tf.losses.sparse_categorical_crossentropy
OPTIMIZER = tf.keras.optimizers.SGD(learning_rate=1e-3) # or use with custom learning rate=> tf.keras.optimizers.SGD(0.02)
METRICS = ["accuracy"]

model_2.compile(loss=LOSS_FUNCTION,
              optimizer=OPTIMIZER,
              metrics=METRICS)

EPOCHS = 10
VALIDATION_SET = (X_valid, y_valid)

new_history = model_2.fit(X_train, y_train, epochs=EPOCHS,
                    validation_data=VALIDATION_SET,batch_size = 32)

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 1.1322 - accuracy: 0.6795 - val_loss: 0.6125 - val_accuracy: 0.8482
Epoch 2/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5742 - accuracy: 0.8556 - val_loss: 0.4342 - val_accuracy: 0.8920
Epoch 3/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4559 - accuracy: 0.8808 - val_loss: 0.3584 - val_accuracy: 0.9058
Epoch 4/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3949 - accuracy: 0.8948 - val_loss: 0.3153 - val_accuracy: 0.9150
Epoch 5/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3546 - accuracy: 0.9041 - val_loss: 0.2862 - val_accuracy: 0.9220
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3273 - accuracy: 0.9103 - val_loss: 0.2652 - val_accuracy: 0.9246
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3040 - accuracy: 0.9167 - val_loss: 0.2472 - val_accuracy:

In [ ]:
model_2.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 42.7451 - accuracy: 0.8436


[42.745121002197266, 0.8435999751091003]

#### Observation
Model 1 performed better than model 2 in the test set. So, we can conclude that Batch Normalization layer after the activation function has some advantage over other approach.

### In test time why not use BN layer ?
We may need to make predictions for individual instances rather than for batches of instances: in this case, we will have no way to compute each input’s mean and standard deviation. Moreover, even if we do have a batch of instances, it may be too small, or the instances may not be independent and identically distributed

### Advantages - 
- Faster convergance 
- Helps to reduce Vanishing and Exploding Gradient Issue.
- It is not affected by choice of activation function.
- No need of scaling

### Disadvantage - 
- Complexity in the network.
- Increase in trainable parameters.
- Slow prediction

### When to use BN ?
- Mostly in CNNs.
- Deep layers more than 16 layers 
- To remove Vanishing and Exploding Gradient Issue in RNNs. 